In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [2]:
# Load the CSV file into a DataFrame
csv_path = 'D:\Omar\Friends\European_HealthCare_Hackathon\ecg_hospitalization\data\processed\meta\data_pairs.csv'  # Replace with the path to your CSV file
df = pd.read_csv(csv_path)

In [3]:
df

,np_file_path,label
0,data\processed\npy\MUSE_20231122_211154_57000.npy,0
1,data\processed\npy\MUSE_20231122_221512_86000.npy,0
2,data\processed\npy\MUSE_20231122_222724_38000.npy,0
3,data\processed\npy\MUSE_20231122_101126_09000.npy,1
4,data\processed\npy\MUSE_20231120_222227_88000.npy,0
...,...,...
23287,data\processed\npy\MUSE_20231120_184943_05000.npy,0
23288,data\processed\npy\MUSE_20231122_222459_73000.npy,0
23289,data\processed\npy\MUSE_20231120_101111_35000.npy,0
23290,data\processed\npy\MUSE_20231122_210819_41000.npy,0


In [4]:
# Initialize empty lists to store data
data_list = []
labels_list = []

In [5]:
# Iterate through rows and load .npy files
for index, row in df.iterrows():
    file_path = row['np_file_path']
    label = row['label']

    # Load the .npy file
    loaded_data = np.load(file_path)

    # Append the loaded data and label to the lists
    data_list.append(loaded_data)
    labels_list.append(label)

In [6]:
# Convert lists to NumPy arrays
data_array = np.array(data_list)
labels_array = np.array(labels_list)

In [7]:
# Print shapes for verification
print("Data Array Shape:", data_array.shape)
print("Labels Array Shape:", labels_array.shape)

Data Array Shape: (23292, 8, 5000)
Labels Array Shape: (23292,)


In [8]:
# Assuming ecg_data is your 3D array of raw ECG data
# Assuming labels are integers (0, 1, 2) representing the three classes
num_patients, num_leads, num_time_points = data_array.shape

In [9]:
#data_array_2d = data_array.reshape(-1, data_array.shape[-1])

In [10]:
X = data_array

In [11]:
y = labels_array.flatten()

In [12]:
X.shape

(23292, 8, 5000)

In [13]:
y.shape

(23292,)

In [14]:
# Set seed for reproducibility
np.random.seed(42)

In [15]:
# Generate random indices for shuffling
indices = np.arange(len(y))
np.random.shuffle(indices)

In [16]:
indices

array([23198,  7131,   167, ...,  5390,   860, 15795])

In [17]:
# Shuffle X and y using the generated indices
X_shuffled = X[indices]
y_shuffled = y[indices]

In [18]:
X = X_shuffled
y = y_shuffled

In [19]:
# Reshape the data to fit the CNN input shape
#X = ecg_data.reshape(num_patients, num_leads, num_time_points)  # Remove the extra dimension
#X = data_array

In [20]:
# Assuming labels are integers (0, 1, 2) representing the three classes
# If not, you might need to preprocess your labels accordingly
#y = labels_array.flatten()

In [21]:
# Convert labels to categorical format
#le = LabelEncoder()
#y_cat = to_categorical(le.fit_transform(y))

In [22]:
#y_cat

In [23]:
y

array([2, 0, 0, ..., 1, 0, 0])

In [24]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.utils.class_weight import compute_sample_weight

In [26]:
unique_classes = np.unique(y_train)

In [27]:
unique_classes

array([0, 1, 2])

In [28]:
class_weights = compute_sample_weight(class_weight='balanced', y=y_train)

In [29]:
class_weights

array([1.14954655, 1.14954655, 1.14954655, ..., 3.51898017, 1.14954655,
       0.54173572])

In [30]:
class_weights.shape

(18633,)

In [31]:
# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(num_leads, num_time_points)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Assuming 3 output classes

In [32]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
#from imblearn.over_sampling import RandomOverSampler

In [34]:
# Train the model
#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [35]:
# Convert labels to categorical format
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train))
y_test = to_categorical(le.fit_transform(y_test))

In [36]:
# Use class_weights in the fit method
model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test), sample_weight=class_weights)

Epoch 1/40


583/583 [==============================] - 6s 9ms/step - loss: 158.9317 - accuracy: 0.3252 - val_loss: 95.8867 - val_accuracy: 0.4134
Epoch 2/40
583/583 [==============================] - 4s 8ms/step - loss: 61.9488 - accuracy: 0.3499 - val_loss: 38.7976 - val_accuracy: 0.3769
Epoch 3/40
583/583 [==============================] - 4s 8ms/step - loss: 22.7345 - accuracy: 0.3863 - val_loss: 14.3624 - val_accuracy: 0.4495
Epoch 4/40
583/583 [==============================] - 5s 8ms/step - loss: 8.3427 - accuracy: 0.4245 - val_loss: 10.7158 - val_accuracy: 0.2756
Epoch 5/40
583/583 [==============================] - 5s 8ms/step - loss: 3.8043 - accuracy: 0.4354 - val_loss: 3.4979 - val_accuracy: 0.3713
Epoch 6/40
583/583 [==============================] - 5s 8ms/step - loss: 2.6117 - accuracy: 0.4330 - val_loss: 2.4052 - val_accuracy: 0.4117
Epoch 7/40
583/583 [==============================] - 5s 8ms/step - loss: 2.3894 - accuracy: 0.4283 - val_loss: 2.3376 - val_accuracy: 0.4

In [39]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

146/146 [==============================] - 0s 3ms/step - loss: 1.1191 - accuracy: 0.0944
Test Loss: 1.1190824508666992
Test Accuracy: 0.09444086998701096
